In [9]:
from src.models import Exponential_Model
from src.criterion import right_censored,RightCensorWrapper
from src.load_data import load_datasets,load_dataframe
from src.utils import train_robust,lower_bound
from src.visualizations import visualize_population_curves_attacked,visualize_individual_curves_attacked

from torch.optim import Adam
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from auto_LiRPA import BoundedModule, BoundedTensor

from tqdm import tqdm
import pandas as pd
import numpy as np
from copy import deepcopy

In [10]:
class ARGS(object):
    def __init__(self):
        pass

In [11]:
args = ARGS()
args.verify=False
args.device="cpu"

args.seed = 123

args.eps=0.5
args.norm=np.inf
args.bound_type = "CROWN-IBP"
args.num_epochs=50
args.lr = 1e-3
args.batch_size= 128
args.scheduler_name = "SmoothedScheduler"
args.scheduler_opts = "start=10,length=20"
args.hidden_dims = [15,15]
args.save_model = ""
args.dataset = "TRACE"


In [12]:
# GOOD DATASETS
# 1. TRACE
# 2. divorce 
# 3. Dialysis
dataset_train,dataset_test = load_datasets(args.dataset,test_size=0.2)

In [13]:
input_dims = dataset_train.tensors[0].shape[1]
output_dim = 1

In [14]:
dataloader_train = DataLoader(dataset_train,batch_size=args.batch_size,shuffle=True)
dataloader_test = DataLoader(dataset_test,batch_size=args.batch_size,shuffle=False)

dataloader_train.mean = dataloader_test.mean = dataset_train.mean
dataloader_train.std = dataloader_test.std = dataset_train.std


dataset_train.tensors[0].shape

torch.Size([1502, 10])

In [15]:
clf_robust = Exponential_Model(input_dim=input_dims,hidden_layers=args.hidden_dims)
clf_fragile = Exponential_Model(input_dim=input_dims,hidden_layers=args.hidden_dims)
clf_fragile.load_state_dict(deepcopy(clf_robust.state_dict()))


# model = BoundedModule(clf, X_train)
model_robust_wrap = BoundedModule(RightCensorWrapper(clf_robust),dataset_train.tensors)
model_fragile_wrap = BoundedModule(RightCensorWrapper(clf_fragile),dataset_train.tensors)

In [16]:
train_robust(model_robust_wrap,dataloader_train,dataloader_test,method="robust",args=args)

C:\Users\lpott\anaconda3\envs\survival\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Epoch 1, learning rate [0.001]
[ 1:   0]: eps=0.00000000 Loss=665.7952 Time=0.0090
[ 1:  10]: eps=0.00000000 Loss=624.1345 Time=0.0087
[ 1:  11]: eps=0.00000000 Loss=613.9793 Time=0.0086
Epoch time: 0.1544, Total time: 0.1544
Evaluating...
[ 1:   2]: eps=0.00000000 Loss=576.3916 Time=0.0040
Epoch 2, learning rate [0.001]
[ 2:   0]: eps=0.00000000 Loss=657.6672 Time=0.0060
[ 2:  10]: eps=0.00000000 Loss=581.3987 Time=0.0056
[ 2:  11]: eps=0.00000000 Loss=573.6357 Time=0.0056
Epoch time: 0.0916, Total time: 0.2461
Evaluating...
[ 2:   2]: eps=0.00000000 Loss=538.8378 Time=0.0030
Epoch 3, learning rate [0.001]
[ 3:   0]: eps=0.00000000 Loss=588.1846 Time=0.0050
[ 3:  10]: eps=0.00000000 Loss=545.3104 Time=0.0052
[ 3:  11]: eps=0.00000000 Loss=535.4977 Time=0.0051
Epoch time: 0.0846, Total time: 0.3307
Evaluating...
[ 3:   2]: eps=0.00000000 Loss=499.1743 Time=0.0033
Epoch 4, learning rate [0.001]
[ 4:   0]: eps=0.00000000 Loss=463.7708 Time=0.0050
[ 4:  10]: eps=0.00000000 Loss=502.2614 T

In [29]:
def input_noise_wrapper(t,e):
    def input_noise(f):
    
        log_exact = e * torch.log(f) + e * -(t * f)
        log_right = (1 - e) * -(f * t)
    
        return (log_exact + log_right).sum()

    return input_noise


In [59]:
def feature_jacobian_wrapper(model):
    def feature_jacobian(x):
        return model(x).sum()

    return feature_jacobian

In [60]:
X_train,T_train,E_train = dataloader_train.dataset.tensors

In [61]:
f = clf_robust(X_train)

In [62]:
import torch

input_noise = input_noise_wrapper(T_train,E_train)
feature_jacobian = feature_jacobian_wrapper(clf_robust)

In [63]:
input_noise_matrix = torch.autograd.functional.hessian(input_noise,f).squeeze()

In [64]:
g = torch.vmap(lambda x: torch.autograd.functional.jacobian(feature_jacobian,x))

AttributeError: module 'torch' has no attribute 'vmap'

In [66]:
feature_jacobian_matrix = torch.autograd.functional.jacobian(feature_jacobian,X_train,vectorize=True)

In [68]:
feature_jacobian_matrix.shape

torch.Size([1502, 10])

In [73]:
GGN = torch.einsum("ij,jk,kl->il",feature_jacobian_matrix.T,input_noise_matrix,feature_jacobian_matrix)

In [38]:
torch.nn.utils.parameters_to_vector(clf_robust.parameters())

tensor([-4.0050e-02, -1.5325e-02,  3.8261e-01, -4.6629e-02,  8.1042e-03,
         7.2691e-02, -2.6946e-02, -2.5298e-01, -5.7435e-02,  6.3444e-02,
        -1.0338e-02, -7.8418e-03, -3.1645e-01,  4.2134e-01,  1.9483e-02,
        -1.0950e-02, -6.5861e-02, -8.8244e-02,  6.8074e-02, -3.2696e-02,
        -1.2317e-01, -2.0445e-01,  1.1611e-01,  1.7681e-01,  3.6831e-01,
        -4.2261e-01, -4.1311e-05,  3.6252e-02, -6.4882e-02,  3.0270e-04,
         1.1336e-03, -2.8301e-01, -5.3418e-02,  1.5328e-01,  1.4185e-01,
        -1.3245e-01, -1.0019e-01, -1.1118e-01,  2.8019e-01,  6.2318e-03,
         8.6517e-03, -2.3194e-02, -1.5506e-01, -1.6453e-01, -3.8632e-01,
         3.0604e-01, -2.7372e-02,  1.2627e-01,  1.9315e-01, -5.6199e-03,
         8.8263e-02, -1.6328e-01,  3.1899e-01, -2.1560e-01,  1.3379e-01,
         8.2785e-02, -8.3005e-02,  7.5767e-02,  8.3884e-02, -1.9764e-03,
         3.9146e-01,  1.9229e-01, -2.4575e-01,  1.4082e-01,  1.7515e-01,
         6.5948e-02,  2.1431e-01,  9.4760e-02,  1.8

In [37]:
input_noise_matrix

tensor([[-157.3515,   -0.0000,   -0.0000,  ...,   -0.0000,   -0.0000,
           -0.0000],
        [  -0.0000,  -33.0515,   -0.0000,  ...,   -0.0000,   -0.0000,
           -0.0000],
        [  -0.0000,   -0.0000,   -0.0000,  ...,   -0.0000,   -0.0000,
           -0.0000],
        ...,
        [  -0.0000,   -0.0000,   -0.0000,  ...,   -0.0000,   -0.0000,
           -0.0000],
        [  -0.0000,   -0.0000,   -0.0000,  ...,   -0.0000,   -0.0000,
           -0.0000],
        [  -0.0000,   -0.0000,   -0.0000,  ...,   -0.0000,   -0.0000,
          -11.7302]])